In [1]:
import os
import re
import shutil
import sys
from datetime import date

import numpy as np
import matplotlib.pyplot as plt

from skimage import io
from PIL import Image
from skimage import measure
from tifffile import imsave

# local imports
from image_arrays import *
from s2p_packer import unpack_hdf
import preprocess as pp

### Activate interactive plotting
By default, inline plots are static. Here we specify one of two options (comment out the undesired command) that will open plots with GUI controls for us.
- **qt ->** figures opened in windows outside the notebook
- **notebook ->** figures within notebook underneath generating cell.

In [2]:
# %matplotlib qt 
%matplotlib notebook

In [3]:
# base_path = "/mnt/Data/prerna_velocity/2021_09_29/"
base_path = "/mnt/Data/prerna_velocity/2021_09_29_processed_4x4/"
control_fname = base_path + "scan2_DD_control_200um_20210929.tif"
blockers_fname = base_path + "scan2_DD_blockers_200um_20210929.tif"

control = io.imread(os.path.join(base_path, control_fname))
blockers = io.imread(os.path.join(base_path, blockers_fname))
stacked = np.concatenate([control, blockers], axis=0)

imsave(os.path.join(base_path, "stacked.tif"), stacked)

In [4]:
movie_plot = StackExplorer(
    stacked,
#     zaxis=recs_xaxis,
    delta=1,
    roi_sz=6,
    vmin=0,
    figsize=(6, 8)
)
movie_plot.ax[1].set_xlabel("Time (s)")
movie_plot.ax[1].set_ylabel("Pixel Value")

print("Movie shape:", stacked.shape)
movie_plot.fig.show()

AttributeError: 'NoneType' object has no attribute 'ndim'

TODO:
- 2d projection of first half and second to make sure the topography is similar in both conditions (no major movement)
- compare variance with mean value (does variance go with elevated baselines?)
- try QSE after subtracting the variance of blocker stack from the control
- still need to try generated glusnfr quanta release variance with rates generated from different QSEs (get an estimate of QSE based on the variance of the sustained phase of poisson quantal sums)